In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import calc_f1
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [2]:
v = '7'
# mn= 'large.lr5e-5.m2'
mn = 'pt.large'
model_dir = f'../working/online/{v}.0/{mn}'

In [3]:
import tensorflow as tf
import torch
from absl import flags
FLAGS = flags.FLAGS
from datasets import Dataset
from src import config
from src.util import *
import melt as mt
import numpy as np
from gezi import tqdm
import gezi
import husky
import lele

In [4]:
argv = open(f'{model_dir}/command.txt').readline().strip().split()
FLAGS(argv)
FLAGS.wandb = False
config.init()
# mt.init()
MAX_LEN = FLAGS.max_len
# TODO for longformer large seems torch can use much larger bs for inference? for torch large+32 use 13.9G 
BATCH_SIZE = 8 if not FLAGS.torch else 32 
model = get_model()
gezi.load_weights(model, model_dir)
try:
  ic(open(f'{model_dir}/path.txt').readline().strip())
except Exception:
  pass

ic| config.py:102 in init_()
    FLAGS.backbone: '../input/allenai/longformer-large-4096'
ic| config.py:111 in init_()- FLAGS.num_classes: 8
ic| config.py:128 in init()
    records_pattern: '../input/feedback-prize-2021/tfrecords/train/*.tfrec'
ic| config.py:131 in init()
    FLAGS.valid_files[:2]: ['../input/feedback-prize-2021/tfrecords/train/70.164.tfrec',
                            '../input/feedback-prize-2021/tfrecords/train/90.164.tfrec']
Some weights of the model checkpoint at ../input/allenai/longformer-large-4096 were not used when initializing LongformerModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializin

In [5]:
inputs = get_test_inputs(FLAGS.backbone)
inputs_ = {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']}

ic| util.py:50 in get_test_inputs()
    len(TEST_IDS): 5
    torch.cuda.is_available(): False
ic| util.py:54 in get_test_inputs()- len(TEST_IDS): 5


  0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
p = gezi.predict(model, inputs_, batch_size=BATCH_SIZE)

Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
list(p.keys())

['pred', 'start_logits']

In [8]:
p.update({
  'id': inputs['id'],
  'word_ids': inputs['word_ids']
})
convert_res(p)

In [9]:
df = get_preds(p)
df

  0%|          | 0/5 [00:00<?, ?it/s]

,id,class,predictionstring
0,D46BCB48440A,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
1,D46BCB48440A,Position,20 21 22 23 24 25 26 27
2,D46BCB48440A,Claim,36 37 38
3,D46BCB48440A,Claim,39 40 41 42
4,D46BCB48440A,Claim,43 45
5,D46BCB48440A,Evidence,56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 7...
6,D46BCB48440A,Evidence,150 151 152 153 154 155 156 157 158 159 160 16...
7,D46BCB48440A,Evidence,223 224 225 226 227 228 229 230 231 232 233 23...
8,D46BCB48440A,Concluding Statement,306 307 308 309 310 311 312 313 314 315 316 31...
9,DF920E0A7337,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...


In [10]:
df.to_csv(f'{model_dir}/submission.csv')